# **Connecting to drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing libraries**

In [ ]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim # Longutude and Latitude
!pip install pycountry_convert
import pycountry_convert as pc
import warnings
warnings.filterwarnings('ignore')

# **Data Aggregation**

### 1. Automated importing and basic cleaning of the 25 dataframes from the website "Our World in Data", preparing them for merging
source: https://ourworldindata.org/

In [161]:
def Our_World_in_Data_CLEANING(csv_address, file_name):
  df = pd.read_csv(csv_address)
  df.rename(columns={'Entity':'Country','Day':'Date'},inplace=True)
  df.drop('Code', axis=1, inplace=True)

  return df

dir = '/content/drive/MyDrive/COVID 19 Data/Our_World_in_Data'

flag = 0
merged_df = pd.DataFrame()
DFcounter = 0
for file in os.listdir(dir):
  
  if file.endswith(".csv"):
    file_name = file
    csv_address = '/content/drive/MyDrive/COVID 19 Data/Our_World_in_Data/'+file_name

    df = Our_World_in_Data_CLEANING(csv_address, file_name)
    #if min(df['Date']) >= '2020-02-17':
    if min(df['Date']) > '2020-02-17':
      continue
    DFcounter+=1
    print(file_name,'countries:', len(df['Country'].unique()))
    print(file_name,'min date',min(df['Date']))
    print(file_name,'max date',max(df['Date']))
    print(' ')
    if flag == 0:
      merged_df =df
      flag = 1
    else:
      merged_df = pd.merge(df, merged_df, on=['Country','Date'])
print(DFcounter)

merged_df['Date'] = pd.to_datetime(merged_df['Date'])
print('[merged_df]')
print('Countries:', len(merged_df['Country'].unique()))
print('Min date',min(merged_df['Date']))
print('Max date',max(merged_df['Date']))
print('Shape',merged_df.shape)

covid-vaccination-policy.csv countries: 187
covid-vaccination-policy.csv min date 2020-01-01
covid-vaccination-policy.csv max date 2022-09-15
 
covid-vaccine-age.csv countries: 186
covid-vaccine-age.csv min date 2020-01-01
covid-vaccine-age.csv max date 2022-09-15
 
face-covering-policies-covid.csv countries: 187
face-covering-policies-covid.csv min date 2020-01-01
face-covering-policies-covid.csv max date 2022-09-15
 
public-campaigns-covid.csv countries: 186
public-campaigns-covid.csv min date 2020-01-01
public-campaigns-covid.csv max date 2022-09-15
 
covid-containment-and-health-index.csv countries: 181
covid-containment-and-health-index.csv min date 2020-01-22
covid-containment-and-health-index.csv max date 2022-09-13
 
school-closures-covid.csv countries: 186
school-closures-covid.csv min date 2020-01-21
school-closures-covid.csv max date 2022-09-15
 
workplace-closures-covid.csv countries: 186
workplace-closures-covid.csv min date 2020-01-01
workplace-closures-covid.csv max date

### 2. Retrieving and cleaning of 3 dataframes 

  
*   Renaming the columns
*   Converting 'Date' to date-time
*   Prepared them for merging 

Source: https://covid19.who.int/data

In [162]:
CumulativeDF = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/WHO/WHO-COVID-19-global-data.csv')
CumulativeDF.rename(columns={'Date_reported':'Date','Country_code':'country code'},inplace=True)
CumulativeDF['Date'] = pd.to_datetime(CumulativeDF['Date'])

LatestDF = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/WHO/WHO-COVID-19-global-table-data.csv')
LatestDF.reset_index(inplace=True)
LatestDF = LatestDF.rename(columns={'index':'Country'})
LatestDF.drop('Deaths - newly reported in last 24 hours',axis=1,inplace=True)
LatestDF.dropna(inplace=True)

VaccineDF = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/WHO/vaccination-data.csv')
VaccineDF = VaccineDF.rename(columns={'COUNTRY':'Country'})
VaccineDF.dropna(inplace=True)

CumulativeDF = CumulativeDF[CumulativeDF['Country'].isin(list(VaccineDF['Country']))]
LatestDF = LatestDF[LatestDF['Country'].isin(list(VaccineDF['Country']))]
VaccineDF = VaccineDF[VaccineDF['Country'].isin(list(CumulativeDF['Country']))] # to assure that they all have the same countries which is 190

CumulativeDF.reset_index(inplace=True, drop=True)
LatestDF.reset_index(inplace=True, drop=True)
VaccineDF.reset_index(inplace=True, drop=True)

print('DF1:',str(len(CumulativeDF['Country'].unique()))+', DF2:',str(len(LatestDF['Country']))+', DF3:',len(VaccineDF['Country']))

DF1: 189, DF2: 189, DF3: 189


In [163]:
merged_df2 = pd.merge(LatestDF, CumulativeDF, on=['Country'])
merged_df2 = pd.merge(merged_df2, VaccineDF, on=['Country'])

merged_df2['Date'] = pd.to_datetime(merged_df2['Date'])
print('merged_df2 countries:', len(merged_df2['Country'].unique()))
print('merged_df2 min date',min(merged_df2['Date']))
print('merged_df2 max date',max(merged_df2['Date']))
print(' ')

print(type(merged_df2.Date[0]))

merged_df2 countries: 189
merged_df2 min date 2020-01-03 00:00:00
merged_df2 max date 2022-09-15 00:00:00
 
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


### Merging the 3 dataframes with the 24 dataframes

In [164]:
merged_df = pd.merge(merged_df2, merged_df, on=['Country','Date'])
merged_df = merged_df.sort_values(by='Date')
print('[merged_df]')
print('Countries:', len(merged_df['Country'].unique()))
print('Min date',min(merged_df['Date']))
print('Max date',max(merged_df['Date']))
print('Shape',merged_df.shape)
print(' ')
merged_df['Date']

[merged_df]
Countries: 102
Min date 2020-02-17 00:00:00
Max date 2022-09-11 00:00:00
Shape (93750, 56)
 


0       2020-02-17
72560   2020-02-17
71636   2020-02-17
6362    2020-02-17
70719   2020-02-17
           ...    
54117   2022-09-11
22146   2022-09-11
18437   2022-09-11
46670   2022-09-11
93749   2022-09-11
Name: Date, Length: 93750, dtype: datetime64[ns]

In [165]:
print('Number of rows =', len(merged_df))
print('Number of columns =', len(merged_df.columns))
print('Number of countries =', len(merged_df['Country'].unique()))
print('Number of rows for each country =',len(merged_df)/len(merged_df['Country'].unique()))
print('With',merged_df.isna().sum().sum(),'Nulls')
print('Number of missing dates in the range between the min and max Dates =',
      len(pd.date_range(start=min(merged_df['Date']), end=max(merged_df['Date'])).difference(merged_df.Date)), '(No gaps in the date column)')

Number of rows = 93750
Number of columns = 56
Number of countries = 102
Number of rows for each country = 919.1176470588235
With 5241 Nulls
Number of missing dates in the range between the min and max Dates = 0 (No gaps in the date column)


In [166]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93750 entries, 0 to 93749
Data columns (total 56 columns):
 #   Column                                                        Non-Null Count  Dtype         
---  ------                                                        --------------  -----         
 0   Country                                                       93750 non-null  object        
 1   Name                                                          93750 non-null  object        
 2   WHO Region                                                    93750 non-null  int64         
 3   Cases - cumulative total                                      93750 non-null  float64       
 4   Cases - cumulative total per 100000 population                93750 non-null  int64         
 5   Cases - newly reported in last 7 days                         93750 non-null  float64       
 6   Cases - newly reported in last 7 days per 100000 population   93750 non-null  int64         
 7   Case

In [167]:
merged_df.reset_index(inplace=True)
merged_df.drop('index', inplace=True, axis=1)

# **Feature Engineering**

### Added Day of the week


*   Monday = 0 
*   Sunday = 6






In [168]:
merged_df['Day of the week'] = list(merged_df['Date'].dt.day_of_week)
merged_df['Day of the week string'] = list(merged_df['Date'].dt.day_name())
merged_df

,Country,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,...,cancel_public_events,workplace_closures,school_closures,containment_index,public_information_campaigns,facial_coverings,vaccine_availability,vaccination_policy,Day of the week,Day of the week string
0,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,17.26,1,0,0,0,0,Monday
1,Kyrgyzstan,Europe,205920,3156.253,85,1.303,0,2991,45.845,0,...,0,0,0,11.31,2,0,0,0,0,Monday
2,Mozambique,Africa,230184,736.461,39,0.125,4,2222,7.109,0,...,0,0,0,5.36,1,0,0,0,0,Monday
3,Australia,Western Pacific,10112229,39655.980,17318,67.914,0,14421,56.553,207,...,0,0,0,22.02,2,0,0,0,0,Monday
4,Zimbabwe,Africa,256939,1728.724,105,0.706,0,5596,37.651,0,...,0,0,0,5.36,1,0,0,0,0,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Latvia,Europe,912002,47806.990,7019,367.935,1419,5965,312.684,9,...,0,0,0,22.62,2,2,3,5,6,Sunday
93746,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,1,1,1,50.48,2,3,0,5,6,Sunday
93747,Israel,Europe,4646872,53686.673,7134,82.421,1112,11667,134.792,2,...,0,0,1,36.90,2,2,3,5,6,Sunday
93748,Guatemala,Americas,1112550,6209.962,6397,35.706,0,19684,109.871,100,...,1,1,2,50.60,2,4,3,5,6,Sunday


### Parsed the "Date" column into three more features 


*   Day
*   Month
*   Year






In [169]:
merged_df['Day'] = pd.to_datetime(merged_df['Date']).dt.day
merged_df['Month'] = pd.to_datetime(merged_df['Date']).dt.month
merged_df['Year'] = pd.to_datetime(merged_df['Date']).dt.year
merged_df

,Country,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,...,containment_index,public_information_campaigns,facial_coverings,vaccine_availability,vaccination_policy,Day of the week,Day of the week string,Day,Month,Year
0,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,17.26,1,0,0,0,0,Monday,17,2,2020
1,Kyrgyzstan,Europe,205920,3156.253,85,1.303,0,2991,45.845,0,...,11.31,2,0,0,0,0,Monday,17,2,2020
2,Mozambique,Africa,230184,736.461,39,0.125,4,2222,7.109,0,...,5.36,1,0,0,0,0,Monday,17,2,2020
3,Australia,Western Pacific,10112229,39655.980,17318,67.914,0,14421,56.553,207,...,22.02,2,0,0,0,0,Monday,17,2,2020
4,Zimbabwe,Africa,256939,1728.724,105,0.706,0,5596,37.651,0,...,5.36,1,0,0,0,0,Monday,17,2,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Latvia,Europe,912002,47806.990,7019,367.935,1419,5965,312.684,9,...,22.62,2,2,3,5,6,Sunday,11,9,2022
93746,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,50.48,2,3,0,5,6,Sunday,11,9,2022
93747,Israel,Europe,4646872,53686.673,7134,82.421,1112,11667,134.792,2,...,36.90,2,2,3,5,6,Sunday,11,9,2022
93748,Guatemala,Americas,1112550,6209.962,6397,35.706,0,19684,109.871,100,...,50.60,2,4,3,5,6,Sunday,11,9,2022


### Added weekend column

In [170]:
names = merged_df['Day of the week']
lst = []
for i in range(len(names)):
  if((names[i] == 5) or (names[i] == 6)):
    lst.append(1)
  else:
    lst.append(0)
    
merged_df['Weekend'] = lst
merged_df[merged_df['Weekend'] == 1]

,Country,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,...,public_information_campaigns,facial_coverings,vaccine_availability,vaccination_policy,Day of the week,Day of the week string,Day,Month,Year,Weekend
510,Switzerland,Europe,4053997,46842.010,7840,90.587,0,13550,156.564,1,...,0,0,0,0,5,Saturday,22,2,2020,1
511,Libya,Eastern Mediterranean,506909,7377.201,31,0.451,20,6437,93.680,0,...,0,0,0,0,5,Saturday,22,2,2020,1
512,Latvia,Europe,912002,47806.990,7019,367.935,1419,5965,312.684,9,...,1,0,0,0,5,Saturday,22,2,2020,1
513,Mexico,Americas,7059936,5475.673,4301,3.336,0,329767,255.767,24,...,0,0,0,0,5,Saturday,22,2,2020,1
514,South Africa,Africa,4014986,6769.642,2174,3.666,262,102129,172.199,0,...,0,0,0,0,5,Saturday,22,2,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Latvia,Europe,912002,47806.990,7019,367.935,1419,5965,312.684,9,...,2,2,3,5,6,Sunday,11,9,2022,1
93746,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,2,3,0,5,6,Sunday,11,9,2022,1
93747,Israel,Europe,4646872,53686.673,7134,82.421,1112,11667,134.792,2,...,2,2,3,5,6,Sunday,11,9,2022,1
93748,Guatemala,Americas,1112550,6209.962,6397,35.706,0,19684,109.871,100,...,2,4,3,5,6,Sunday,11,9,2022,1


### Adding death ratio per country


> Death Ratio = total number of deaths / total number of cases

In [171]:
DeathRatioDF = (merged_df.groupby('Country')['Deaths - cumulative total'].max()/merged_df.groupby('Country')['Cases - cumulative total'].max())*100
DeathRatioDF = DeathRatioDF.to_frame()
DeathRatioDF.reset_index(inplace=True)
DeathRatioDF = DeathRatioDF.rename(columns={0:'Death Ratio'})

merged_df = pd.merge(merged_df, DeathRatioDF, on=['Country'])
merged_df

,Country,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,...,facial_coverings,vaccine_availability,vaccination_policy,Day of the week,Day of the week string,Day,Month,Year,Weekend,Death Ratio
0,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,0,Monday,17,2,2020,0,1.186643
1,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,1,Tuesday,18,2,2020,0,1.186643
2,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,2,Wednesday,19,2,2020,0,1.186643
3,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,3,Thursday,20,2,2020,0,1.186643
4,India,South-East Asia,44516479,3225.822,37843,2.742,6422,528250,38.279,160,...,0,0,0,4,Friday,21,2,2020,0,1.186643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,3,0,5,2,Wednesday,7,9,2022,0,5.237242
93746,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,3,0,5,3,Thursday,8,9,2022,0,5.237242
93747,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,3,0,5,4,Friday,9,9,2022,0,5.237242
93748,Peru,Americas,4127612,12518.592,8820,26.750,1150,216173,655.629,191,...,3,0,5,5,Saturday,10,9,2022,1,5.237242


### Adding continents and droping the name column

In [172]:
continents = []
countries = merged_df['Country'].unique()

for country in countries:
  country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
  continent_name = pc.country_alpha2_to_continent_code(country_code)
  continents.append(continent_name)

continentsDic = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

ContinentDF = pd.DataFrame(list(zip(countries,continents)), columns=['Country','Continent'])

for i in range(len(ContinentDF['Continent'])):
  ContinentDF['Continent'][i] = continentsDic[ContinentDF['Continent'][i]]
  

ContinentDF

,Country,Continent
0,India,Asia
1,Kyrgyzstan,Asia
2,Mozambique,Africa
3,Australia,Australia
4,Zimbabwe,Africa
...,...,...
97,Finland,Europe
98,Norway,Europe
99,Costa Rica,North America
100,Yemen,Asia


In [173]:
merged_df = pd.merge(merged_df, ContinentDF, on=['Country'])
merged_df.drop('Name', axis=1,inplace=True)
merged_df

,Country,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,...,vaccine_availability,vaccination_policy,Day of the week,Day of the week string,Day,Month,Year,Weekend,Death Ratio,Continent
0,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,0,0,Monday,17,2,2020,0,1.186643,Asia
1,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,0,1,Tuesday,18,2,2020,0,1.186643,Asia
2,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,0,2,Wednesday,19,2,2020,0,1.186643,Asia
3,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,0,3,Thursday,20,2,2020,0,1.186643,Asia
4,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,0,4,Friday,21,2,2020,0,1.186643,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,0,5,2,Wednesday,7,9,2022,0,5.237242,South America
93746,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,0,5,3,Thursday,8,9,2022,0,5.237242,South America
93747,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,0,5,4,Friday,9,9,2022,0,5.237242,South America
93748,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,0,5,5,Saturday,10,9,2022,1,5.237242,South America


### Added Longutude and Latitude

In [175]:
geolocator = Nominatim()
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

countries = merged_df['Country'].unique()
lat=[]
lng=[]
for country in countries:
  lat.append(geolocate(country)[0])
  lng.append(geolocate(country)[1])

CountryDF = pd.DataFrame(list(zip(countries,lat,lng)), columns=['Country','latitude','longitude'])
CountryDF

,Country,latitude,longitude
0,India,22.351115,78.667743
1,Kyrgyzstan,41.508932,74.724091
2,Mozambique,-19.302233,34.914498
3,Australia,-24.776109,134.755000
4,Zimbabwe,-18.455496,29.746841
...,...,...,...
97,Finland,63.246778,25.920916
98,Norway,60.500021,9.099972
99,Costa Rica,10.273563,-84.073910
100,Yemen,16.347124,47.891527


In [176]:
merged_df = pd.merge(merged_df, CountryDF, on=['Country'])
print('With',merged_df.isna().sum().sum(),'Nulls')
merged_df

With 5241 Nulls


,Country,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,...,Day of the week,Day of the week string,Day,Month,Year,Weekend,Death Ratio,Continent,latitude,longitude
0,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,Monday,17,2,2020,0,1.186643,Asia,22.351115,78.667743
1,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1,Tuesday,18,2,2020,0,1.186643,Asia,22.351115,78.667743
2,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,2,Wednesday,19,2,2020,0,1.186643,Asia,22.351115,78.667743
3,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,3,Thursday,20,2,2020,0,1.186643,Asia,22.351115,78.667743
4,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,4,Friday,21,2,2020,0,1.186643,Asia,22.351115,78.667743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93745,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,2,Wednesday,7,9,2022,0,5.237242,South America,-6.869970,-75.045851
93746,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,3,Thursday,8,9,2022,0,5.237242,South America,-6.869970,-75.045851
93747,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,4,Friday,9,9,2022,0,5.237242,South America,-6.869970,-75.045851
93748,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5,Saturday,10,9,2022,1,5.237242,South America,-6.869970,-75.045851


In [177]:
merged_df.dropna(inplace=True)
merged_df.reset_index(inplace=True)
merged_df.drop('index',axis=1,inplace=True)
merged_df

,Country,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,...,Day of the week,Day of the week string,Day,Month,Year,Weekend,Death Ratio,Continent,latitude,longitude
0,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,0,Monday,17,2,2020,0,1.186643,Asia,22.351115,78.667743
1,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1,Tuesday,18,2,2020,0,1.186643,Asia,22.351115,78.667743
2,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,2,Wednesday,19,2,2020,0,1.186643,Asia,22.351115,78.667743
3,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,3,Thursday,20,2,2020,0,1.186643,Asia,22.351115,78.667743
4,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,4,Friday,21,2,2020,0,1.186643,Asia,22.351115,78.667743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91461,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,2,Wednesday,7,9,2022,0,5.237242,South America,-6.869970,-75.045851
91462,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,3,Thursday,8,9,2022,0,5.237242,South America,-6.869970,-75.045851
91463,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,4,Friday,9,9,2022,0,5.237242,South America,-6.869970,-75.045851
91464,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5,Saturday,10,9,2022,1,5.237242,South America,-6.869970,-75.045851


### Adding population, area, and density of each country.

In [178]:
PopulationDF = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/World_Population_Review/population-area-density.csv')
PopulationDF.rename(columns={'name':'Country'}, inplace=True)
PopulationDF

,Rank,Country,pop2022,pop2021,GrowthRate,area,Density
0,1,China,1425887.337,1425893.465,1.0000,9706961,146.8933
1,2,India,1417173.173,1407563.842,1.0068,3287590,431.0675
2,3,United States,338289.857,336997.624,1.0038,9372610,36.0935
3,4,Indonesia,275501.339,273753.191,1.0064,1904569,144.6529
4,5,Pakistan,235824.862,231402.117,1.0191,881912,267.4018
...,...,...,...,...,...,...,...
205,206,San Marino,33.660,33.745,0.9975,61,551.8033
206,207,Palau,18.055,18.024,1.0017,459,39.3355
207,208,Nauru,12.668,12.511,1.0125,21,603.2381
208,209,Tuvalu,11.312,11.204,1.0096,26,435.0769


In [179]:
merged_df = pd.merge(merged_df, PopulationDF, on=['Country'])
merged_df

,Country,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,...,Death Ratio,Continent,latitude,longitude,Rank,pop2022,pop2021,GrowthRate,area,Density
0,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1.186643,Asia,22.351115,78.667743,2,1417173.173,1407563.842,1.0068,3287590,431.0675
1,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1.186643,Asia,22.351115,78.667743,2,1417173.173,1407563.842,1.0068,3287590,431.0675
2,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1.186643,Asia,22.351115,78.667743,2,1417173.173,1407563.842,1.0068,3287590,431.0675
3,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1.186643,Asia,22.351115,78.667743,2,1417173.173,1407563.842,1.0068,3287590,431.0675
4,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,1.186643,Asia,22.351115,78.667743,2,1417173.173,1407563.842,1.0068,3287590,431.0675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90584,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5.237242,South America,-6.869970,-75.045851,45,34049.588,33715.471,1.0099,1285216,26.4933
90585,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5.237242,South America,-6.869970,-75.045851,45,34049.588,33715.471,1.0099,1285216,26.4933
90586,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5.237242,South America,-6.869970,-75.045851,45,34049.588,33715.471,1.0099,1285216,26.4933
90587,Peru,4127612,12518.592,8820,26.750,1150,216173,655.629,191,0.579,...,5.237242,South America,-6.869970,-75.045851,45,34049.588,33715.471,1.0099,1285216,26.4933


In [180]:
print('Number of rows =', len(merged_df))
print('Number of columns =', len(merged_df.columns))
print('Number of countries =', len(merged_df['Country'].unique()))
print('Number of rows for each country =',len(merged_df)/len(merged_df['Country'].unique()))
print('With',merged_df.isna().sum().sum(),'Nulls')
print('Number of missing dates in the range between the min and max Dates =',
      len(pd.date_range(start=min(merged_df['Date']), end=max(merged_df['Date'])).difference(merged_df.Date)), '(No gaps in the date column)')

Number of rows = 90589
Number of columns = 71
Number of countries = 99
Number of rows for each country = 915.040404040404
With 0 Nulls
Number of missing dates in the range between the min and max Dates = 0 (No gaps in the date column)


### Adding
*   Life expectancy
*   Sub-region
*   Income group
*   Rural percentage
*   Temperature
*   Time zone
*   Poverty ratio
*   Health care index

**Important: Some column names and formating were changed using Excel before making the csv! Make sure to change the column names in Excel after downloading the data from the sources we've listed. If you struggle to obtain the data just email us or contact us on LinkedIn and we will send all the data.**

In [181]:
life = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/Worldometers/life_expectancy.csv')
sub_region = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/GitHub/sub-region.csv')
income = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/The_World_Bank/income.csv')
rural = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/The_World_Bank/rural-percentage.csv')
temp = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/listfist/temperature.csv')


time_zone = pd.read_excel('/content/drive/MyDrive/COVID 19 Data/Wikipedia/time-zone.xlsx')
time_zone.dropna(inplace=True)
time_zone.reset_index(inplace=True)
time_zone.drop('index', axis=1, inplace=True)
time_zone = time_zone.astype({"No. of time zones": int})
time_zone['country'] = time_zone['country'].str.strip()

poverty = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/World_Population_Review/poverty.csv')
poverty.rename(columns={'percPoverty':'poverty percentage '}, inplace=True)

#health = pd.read_csv('/content/drive/MyDrive/COVID 19 Data/Numbeo/health-care.csv')
health = pd.read_csv('/content/csvData (6).csv')

**column names after changing them using Excel**

In [182]:
print('life expectancy columns: ',list(life.columns))
print('sub_region columns: ',list(sub_region.columns))
print('income columns: ',list(income.columns))
print('rural columns: ',list(rural.columns))
print('temperature columns: ',list(temp.columns))
print('time_zone columns: ',list(time_zone.columns))
print('poverty columns: ',list(poverty.columns))
print('Health care columns: ',list(health.columns))

life expectancy columns:  ['country', 'Life Expectancy(both sexes) ', 'Life Expectancy(Females)', 'Life Expectancy(Males)']
sub_region columns:  ['country', 'sub-region']
income columns:  ['country', 'Income group']
rural columns:  ['country', 'country code', 'rural percentage(2021)']
temperature columns:  ['country', 'Average Temperature(1991-2020) C', 'Coldest Month(1991-2020) C', 'Hottest Month(1991-2020) C', 'Variation(1991-2020) C']
time_zone columns:  ['country', 'No. of time zones', 'Time zone']
poverty columns:  ['country', 'poverty percentage ']
Health care columns:  ['country', 'legatumRank2020', 'legatumRank2019', 'ceoworldRank']


In [183]:
rural.drop('country code', axis=1, inplace=True)
merged_df.rename(columns={'Country':'country'}, inplace=True)

In [184]:
merged_df = pd.merge(merged_df, life, on=['country'])
merged_df = pd.merge(merged_df, sub_region, on=['country'])
merged_df = pd.merge(merged_df, income, on=['country'])
merged_df = pd.merge(merged_df, rural, on=['country'])
merged_df = pd.merge(merged_df, temp, on=['country'])
merged_df = pd.merge(merged_df, time_zone, on=['country'])
merged_df = pd.merge(merged_df, poverty, on=['country'])
merged_df = pd.merge(merged_df, health, on=['country'])

In [185]:
print('Number of rows =', len(merged_df))
print('Number of columns =', len(merged_df.columns))
print('Number of countries =', len(merged_df['country'].unique()))
print('Number of rows for each country =',len(merged_df)/len(merged_df['country'].unique()))
print('With',merged_df.isna().sum().sum(),'Nulls')
print('Number of missing dates in the range between the min and max Dates =',
      len(pd.date_range(start=min(merged_df['Date']), end=max(merged_df['Date'])).difference(merged_df.Date)), '(No gaps in the date column)')

Number of rows = 76831
Number of columns = 87
Number of countries = 84
Number of rows for each country = 914.6547619047619
With 26363 Nulls
Number of missing dates in the range between the min and max Dates = 0 (No gaps in the date column)


# Cleaning


*   Renamed some Countries 
*   Droppped a country with null data "Turkmenistan"
*   Dropped some redundant features ( "country code, ”   who_region.1", “who region”, "rank", "data_source") 
*   Dropped missing values <1k
*   Dropped “ceoworldRank” feature because it has 24k missing values around 32% of the entire dataset
*   Converted all feature names into lower case 








In [186]:
merged_df['country'] = merged_df['country'].replace('Israel', 'Palestine')

In [187]:
merged_df = merged_df[merged_df.country != 'Turkmenistan']
merged_df = merged_df.sort_values(by='Date')

In [188]:
for column in merged_df:
  merged_df.rename(columns={column:column.lower()}, inplace=True)
merged_df

,country,who region,cases - cumulative total,cases - cumulative total per 100000 population,cases - newly reported in last 7 days,cases - newly reported in last 7 days per 100000 population,cases - newly reported in last 24 hours,deaths - cumulative total,deaths - cumulative total per 100000 population,deaths - newly reported in last 7 days,...,average temperature(1991-2020) c,coldest month(1991-2020) c,hottest month(1991-2020) c,variation(1991-2020) c,no. of time zones,time zone,poverty percentage,legatumrank2020,legatumrank2019,ceoworldrank
0,India,44516479,3225.822,37843,2.742,6422,528250,38.279,160,0.012,...,24.68,16.3,31.1,14.8,1,UTC+05:30 (IST),21.9,101,98,19.0
57460,Latvia,912002,47806.990,7019,367.935,1419,5965,312.684,9,0.472,...,6.86,-10.9,21.0,31.9,1,UTC+02:00 (EET),22.9,32,32,48.0
56528,Paraguay,715806,10035.782,237,3.323,0,19530,273.816,36,0.505,...,23.73,15.0,28.9,13.9,1,UTC−04:00,23.5,79,78,NaN
55597,Malaysia,4811327,14865.375,13890,42.915,2431,36296,112.142,34,0.105,...,25.75,24.5,27.2,2.7,1,UTC+08:00 (Malaysian Standard Time),5.6,40,42,34.0
54690,Kazakhstan,1482094,7893.258,1232,6.561,181,19047,101.440,4,0.021,...,6.91,-18.0,25.5,43.5,2,"UTC+05:00 — western Kazakhstan (Aktobe, Atyrau...",4.3,62,66,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60220,Poland,6233117,16421.029,30681,80.829,5858,117316,309.067,97,0.256,...,8.85,-6.7,21.3,28.0,1,UTC+01:00 (CET),15.4,36,36,51.0
58397,Latvia,912002,47806.990,7019,367.935,1419,5965,312.684,9,0.472,...,6.86,-10.9,21.0,31.9,1,UTC+02:00 (EET),22.9,32,32,48.0
61157,Uruguay,982846,28293.679,1660,47.787,0,7462,214.812,7,0.202,...,17.93,8.8,25.8,17.0,1,UTC−03:00,8.8,39,39,39.0
52870,Canada,4197701,11122.049,18365,48.659,0,44347,117.500,237,0.628,...,-6.36,-27.6,12.8,40.4,6,UTC−08:00 (PT) — larger western part of Britis...,11.6,14,13,23.0


In [189]:
merged_df.ceoworldrank.isna().sum()

24531

In [190]:
merged_df.drop(['country code','rank', 'data_source','vaccine_availability','ceoworldrank','who region'],axis=1,inplace=True)

In [191]:
print('Missing Vlaues: ',merged_df.isna().sum().sum())
merged_df.dropna(inplace=True)

Missing Vlaues:  1832


In [193]:
print('Number of rows =', len(merged_df))
print('Number of columns =', len(merged_df.columns))
print('Number of countries =', len(merged_df['country'].unique()))
print('Number of rows for each country =',len(merged_df)/len(merged_df['country'].unique()))
print('With',merged_df.isna().sum().sum(),'Nulls')
print('Number of missing dates in the range between the min and max Dates =',
      len(pd.date_range(start=min(merged_df['date']), end=max(merged_df['date'])).difference(merged_df.date)), '(No gaps in the date column)')

Number of rows = 74999
Number of columns = 81
Number of countries = 82
Number of rows for each country = 914.6219512195122
With 0 Nulls
Number of missing dates in the range between the min and max Dates = 0 (No gaps in the date column)


# Saving the DataFrame as csv.

In [194]:
merged_df = merged_df.sort_values(by='date')
merged_df.to_csv('COVID19_Data.csv')

In [195]:
merged_df.isnull().sum().sum()

0

In [196]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74999 entries, 0 to 76830
Data columns (total 81 columns):
 #   Column                                                        Non-Null Count  Dtype         
---  ------                                                        --------------  -----         
 0   country                                                       74999 non-null  object        
 1   cases - cumulative total                                      74999 non-null  float64       
 2   cases - cumulative total per 100000 population                74999 non-null  int64         
 3   cases - newly reported in last 7 days                         74999 non-null  float64       
 4   cases - newly reported in last 7 days per 100000 population   74999 non-null  int64         
 5   cases - newly reported in last 24 hours                       74999 non-null  int64         
 6   deaths - cumulative total                                     74999 non-null  float64       
 7   deat

In [197]:
print('Number of rows =', len(merged_df))
print('Number of columns =', len(merged_df.columns))
print('Number of countries =', len(merged_df['country'].unique()))
print('Number of rows for each country =',len(merged_df)/len(merged_df['country'].unique()))
print('With',merged_df.isna().sum().sum(),'Nulls')
print('Number of missing dates in the range between the min and max Dates =',
      len(pd.date_range(start=min(merged_df['date']), end=max(merged_df['date'])).difference(merged_df.date)), '(No gaps in the date column)')

Number of rows = 74999
Number of columns = 81
Number of countries = 82
Number of rows for each country = 914.6219512195122
With 0 Nulls
Number of missing dates in the range between the min and max Dates = 0 (No gaps in the date column)
